# Limpieza de carpetas de investigación del 2023

Estos datos corresponden al periodo del 01/Enero/2023 al 31/Marzo/2023

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.impute import SimpleImputer
import math
import re

## Entendimiento de los datos

In [ ]:
df = pd.read_csv('data/da_2023.csv')
df.shape

In [ ]:
df.columns

In [ ]:
#df[np.logical_and(df["colonia"].isnull(), df["persona"] == "MORAL")]

In [ ]:
df.dtypes

In [ ]:
df.nunique()

In [ ]:
df["Año_hecho"].describe()

### Tenemos delitos cometidos desde el año **1962** a los cuáles se les abrió carpetas de investigación hasta el año **2023**

In [ ]:
df["Año_hecho"].unique()

In [ ]:
df["Categoria"].unique()

In [ ]:
df.loc[df["Año_hecho"]==1962]

In [ ]:
df["competencia"].unique()

### Resumen del entendimiento de los datos

1. Existen carpetas de investigación sobre delitos cometidos desde 1962, abiertas hasta el 2023.

2. Tenemos **16 categorías de delitos**.

3. Hay **5 tipos de calidades jurídicas**.

4. Describiendo los **3 competencias de delitos**:
    
    * Fuero común: Aquellos delitos que afectas directamente a la persona.
    * Hechos no delictivos: No son delitos.
    * Incompetencias: La persona es incapaz de ser juzgada o de ser responsable de sus acciones debido a condiciones físicas o mentales.
    
    
    
5. En la columna de alcaldías estamos considerando un **extra llamado 'fuera de la ciudad de méxico'**

6. Tenemos **255 tipos de delitos**.

## Limpieza

1. [X] Eliminar columnas que se relacionen con el tiempo en que se hizo la denuncia
2. [X] Estandarizar a minusculas las columnas restantes
3. [X] Renombrar columnas a 1 sola palabra
4. [X] Eliminar la palabra hecho de todas las columnas que la contengan
5. [X] Convertir a formato de fecha si corresponde
6. [X] Eliminar la columna de Municipio ya que solo vamos a analizar dentro de CDMX
7. [ ] Rellenar los NaN  

        7.1 [X] 'sexo' no, hasta tener separados los dataframes  
        
        7.2 [ ] 'edad' no, hasta tener separados los dataframes  
        
        7.3 [X] 'persona' no, eliminar los registros que no tengan este campo  
        
        7.4 [X] 'año, mes, fecha y hora' no, son 7 entonces eliminemos  
        
        7.5 [ ] 'colonia' si, quizas teniendo en cuenta las colonias que mas se repiten por alcaldia  
        
        7.6 [X] 'latitud y longitud' borrar columnas (realmente no se usarán)
        
8. [X] Cambiar a enteros los años
9. [X] Si existe un tipo de dato de "hora", cambiarlo en la que corresponda  

        9.1 [X] Eliminar aquellos registros con categoria "hecho no delictivo" y calidad "incompetencias"
        
10. [X] Separar en 2 DataFrames teniendo en cuenta el tipo de persona
11. [X] Del dataframe de personas morales, eliminar las columnas de sexo, edad, tipopersona
12. [ ] En la columna de edades de personas físicas, llenar los NA con algún **algoritmo de imputación**



### Paso 1

In [ ]:
df.drop(
        columns=["fgj_colonia_registro", "idCarpeta", "Año_inicio", "Mes_inicio", "FechaInicio", "HoraInicio"], 
        inplace=True
        )

### Paso 2

In [ ]:
df.rename(columns= lambda x: x.lower(), inplace=True)

### Paso 3

In [ ]:
df.rename(columns = 
         {
             'tipopersona': 'persona',
             'calidadjuridica': 'calidad',
             'colonia_datos': 'colonia'
         }, 
        inplace=True)

### Paso 4

In [ ]:
def find(name):
    return re.sub(r'_?hechos?$', '', name)

df.columns = df.columns.to_frame()[0].agg(find)

### Paso 5

In [ ]:
df.fecha = pd.to_datetime(df.fecha)

### Paso 6: Eliminar Municipio

In [ ]:
df.drop(columns="municipio", inplace=True)

### Paso 7.3, 7.4

In [ ]:
# Borrar persona, año, mes fecha y hora nulos
df.dropna(subset=["persona", "año", "mes", "fecha", "hora"], inplace=True)

### Paso 7.6

In [ ]:
df.drop(columns=["latitud", "longitud"], inplace=True)

### Paso 8

In [ ]:
df["año"] = df['año'].astype(int)

### Paso 9

In [ ]:
df["hora"] = pd.to_datetime(df['hora']).dt.time

### Paso 9.1

In [ ]:
df = df[np.logical_and(df["categoria"] != "HECHO NO DELICTIVO", df["calidad"] != "INCOMPETENCIAS")]

### Paso 10: separación de dataframe

In [ ]:
df_fisica = df.query('persona == "FISICA"')
df_moral = df.query('persona == "MORAL"')

### Paso 11

In [ ]:
df_moral = df_moral.drop(columns = ["edad", "sexo", "persona"])
df_moral.head(1)

### Paso 12

In [ ]:
df_fisica.isna().sum()

In [ ]:
df_moral.isna().sum()

### Rellenado de colonias en moral

Agrupar por alcaldía y colonia

Seleccionar las mas ocurrentes en cada alcaldía y rellenarlo

In [ ]:
# BORRAR REGISTROS QUE EN ALCALDIA CONTENGA CDMX Y FUERA DE CDMX
df_moral = df_moral[np.logical_and(df_moral["alcaldia"] != "CDMX", df_moral["alcaldia"] != "FUERA DE CDMX")]

In [ ]:
imputer = SimpleImputer(strategy='most_frequent')

def imputacion(x):
    #print(x.values.reshape(-1, 1))
    return imputer.fit_transform(x.values.reshape(-1, 1)).ravel()

In [ ]:
df_moral['colonia'] = df_moral.groupby('alcaldia')['colonia'].transform(imputacion)

In [ ]:
df_moral.isna().sum()

### Imputación de las personas físicas

In [ ]:
df_fisica.isna().count()

In [ ]:
df_fisica.groupby("alcaldia")["colonia"].count()

In [ ]:
df_fisica = df_fisica[np.logical_and(df_fisica["alcaldia"] != "CDMX", df_fisica["alcaldia"] != "FUERA DE CDMX")]

In [ ]:
df_fisica['colonia'] = df_fisica.groupby('alcaldia')['colonia'].transform(imputacion)

In [ ]:
df_fisica.isna().sum()

### Edades

In [ ]:
medias_edades = df_fisica.groupby(by=["delito"])["edad"].mean().reset_index()
medias_edades.columns = ["delitos", "media_edad"]
#medias_edades[medias_edades["delitos"] == "ROBO DE OBJETOS"]

In [ ]:
delitos_nulos = medias_edades[medias_edades["media_edad"].isna()]
delitos_completos = medias_edades[~medias_edades["media_edad"].isna()]

In [ ]:
delitos_borrar = [x for x, row in delitos_nulos.iterrows()]

In [ ]:
df_fisica = df_fisica[~df_fisica['delito'].isin(delitos_nulos)]

In [ ]:
for index, row in df_fisica[["delito", "edad"]].iterrows():
    if not pd.notnull(row["edad"]):
        media_edad = medias_edades[medias_edades["delitos"] == row["delito"]]["media_edad"]
        df_fisica.at[index, "edad"] = media_edad


In [ ]:
df_fisica.isna().sum()

In [ ]:
df_fisica.dropna(inplace=True)

In [ ]:
df_fisica["edad"] = df_fisica["edad"].astype(int)

In [ ]:
df_fisica.drop(columns="persona", inplace = True)

In [ ]:
df_fisica.isna().sum()

### Exportando dataframes limpios

In [ ]:
df_fisica.to_csv('data/fisicas.csv', index=False)
df_fisica

In [ ]:
df_moral.to_csv('data/morales.csv')